In [8]:
import cv2
import blob
import cutEye as ce
import time 

In [9]:
isToCut = True

print("*Olá! Eu sou um chatbot*.\nTenho apenas uma pergunta. Você está em um ambiente sem luz?[S/n]")
print(">", end=' ')
inp = input()
print("\nPress the keyword 'Q' to exit or 'P' to print the time.\n")

if inp.upper() == 'S':
    isToCut =  False


*Olá! Eu sou um chatbot*.
Tenho apenas uma pergunta. Você está em um ambiente sem luz?[S/n]
> n

Press the keyword 'Q' to exit or 'P' to print the time.



In [11]:
LIM = 100
ll = -1
lr = -1

tbegin = time.time()
print("Tempo inicial: ", tbegin)
title_window = ''

Tempo inicial:  1626392491.2206092


In [12]:
def on_trackbar(val):
    alpha = val / alpha_slider_max
    beta = (1.0 - alpha)
    dst = cv2.addWeighted(src1, alpha, src2, beta, 0.0)
    cv2.imshow(title_window, dst)

def nothing(x):
    pass

In [13]:
scaleFactor = 1.3
minNeighbors = 5

name = "Eye tracking"

bar_nameR = "value of first trackbar:"
bar_nameL = "value of second trackbar:"

In [14]:
cam = cv2.VideoCapture("/dev/video2")
cam.set(cv2.CAP_PROP_FPS, 30)

True

In [15]:
limite = 0

#importa o classificador de face
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
#importa o classificador de olho
eye_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_eye.xml')

In [16]:
#Iniciando o algoritmo Blob
blob_parm = cv2.SimpleBlobDetector_Params()
blob_parm.filterByArea = True
blob_parm.maxArea = 1500 #unidade em pixels
blob_dt = cv2.SimpleBlobDetector_create(blob_parm)

#Desenha as barras deslizantes na janela
cv2.namedWindow(name)
cv2.createTrackbar(bar_nameR, name, 0, 255, nothing)
cv2.createTrackbar(bar_nameL, name, 0, 255, nothing)

while(True):
    ret, frame = cam.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor, minNeighbors)
    
    for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            cv2.GaussianBlur(roi_gray, (3, 3), 0.005 * (w*h))

            #aplica o algoritmo de detecção do olho
            eyes = eye_cascade.detectMultiScale(roi_gray)
            for i in range(len(eyes)):    
                (ex,ey,ew,eh) = eyes[i]
                if ey < 1/2*(y+h) - 100:
                    cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)   
                    try:
                        limiteR = cv2.getTrackbarPos(bar_nameR, name) 
                        limiteL = cv2.getTrackbarPos(bar_nameL, name) 
                    except IndexError:
                        print("Há mais de um elemento na cena que está atrapalhando a detecção do usuário. Try Again Pls :)")

                    eye_img = roi_color[ey:ey+eh, ex:ex+ew]
                    if isToCut:
                        eye_img = ce.cut_eyebrows(eye_img)

                    keypoints = blob.blob_process(eye_img, blob_dt, limiteR)
                    keypoints1 = blob.blob_process(eye_img, blob_dt, limiteL)
                    cv2.drawKeypoints(eye_img, keypoints, eye_img, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                    cv2.drawKeypoints(eye_img, keypoints1, eye_img, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                    #cv2.imshow("asdf",blob.img_blob_process(eye_img, blob_dt, limite, 1))
                    
                    for i in keypoints:
                        #print(ex + i.pt[0], ey + i.pt[1])
                        if LIM < ex:
                            print("E ", end='')
                            ll = ex
                        elif LIM > ex:
                            print("D ", end='')
                            lr = ex
                        if ll!=-1 and lr!=-1:
                            LIM = (lr+ll)/2
                                                    
                        print(ex + i.pt[0], ey + i.pt[1], end=' ')
                        print(time.time() - tbegin)
        
                    for i in keypoints1:
                        #print(ex + i.pt[0], ey + i.pt[1])
                        if LIM < ex:
                            print("E ", end='')
                            ll = ex
                        elif LIM > ex:
                            print("D ", end='')
                            lr = ex
                        if ll!=-1 and lr!=-1:
                            LIM = (lr+ll)/2
                                                    
                        print(ex + i.pt[0], ey + i.pt[1], end=' ')
                        print(time.time() - tbegin)
    
    cv2.imshow(name, frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
        
cam.release()
cv2.destroyWindow(name) 

[ WARN:0] global /tmp/pip-req-build-xw6jtoah/opencv/modules/highgui/src/window.cpp (703) createTrackbar UI/Trackbar(value of first trackbar:@Eye tracking): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.
[ WARN:0] global /tmp/pip-req-build-xw6jtoah/opencv/modules/highgui/src/window.cpp (703) createTrackbar UI/Trackbar(value of second trackbar:@Eye tracking): Using 'value' pointer is unsafe and deprecated. Use NULL as value pointer. To fetch trackbar value setup callback.


E 214.9466667175293 93.00666618347168 12.121406078338623
D 71.2933349609375 96.0 13.272436380386353
D 69.31963348388672 95.65296745300293 13.431221008300781
D 68.5 87.0 13.522406816482544
E 202.7325096130371 85.18930053710938 18.62814450263977
E 206.87461853027344 88.37003135681152 18.886123180389404
E 204.63580322265625 83.01234531402588 19.66576886177063
E 202.6564884185791 82.99745559692383 20.19532060623169
E 201.53448295593262 75.5 20.277459144592285
E 199.98333358764648 80.9583330154419 24.527440786361694
E 195.72631645202637 81.1192979812622 24.720149040222168
E 203.69523811340332 86.5206356048584 25.171595811843872
E 206.0 89.5 25.256622791290283
E 208.53703689575195 86.76851844787598 25.337833404541016
